In [ ]:
# Cloner votre dépôt GitHub
!git clone https://github.com/yousef-elgarch1/WebSecure.git


Cloning into 'WebSecure'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 49 (delta 8), reused 49 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 339.86 KiB | 3.86 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
%cd WebSecure



/content/WebSecure


#WEBSECURE : WAEBISTE

In [ ]:
# Installation des dépendances
!pip install flask flask-cors requests beautifulsoup4 pyngrok python-dateutil

In [5]:
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import json
from datetime import datetime
import subprocess
import threading
import time
import requests

# Import your custom modules - make sure these exist in the same directory
from website_info_extractor import WebsiteInfoExtractor
from security_analyzer import SecurityAnalyzer
from risk_predictor import RiskPredictor

app = Flask(__name__)
# Enable CORS for all origins - this is important for API access
CORS(app, resources={r"/api/*": {"origins": "*"}})

# Create necessary directories
os.makedirs('reports', exist_ok=True)
os.makedirs('models', exist_ok=True)

# Initialize analyzers
info_extractor = WebsiteInfoExtractor()
security_analyzer = SecurityAnalyzer()
risk_predictor = RiskPredictor()

@app.route('/api/health', methods=['GET'])
def health_check():
    return jsonify({
        'status': 'healthy',
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'version': '1.0.0'
    })

@app.route('/api/analyze', methods=['POST'])
def analyze_website():
    data = request.get_json()

    if not data:
        return jsonify({'error': 'Invalid JSON data'}), 400

    url = data.get('url')
    scan_type = data.get('scan_type', 'quick')

    if not url:
        return jsonify({'error': 'URL is required'}), 400

    try:
        # Step 1: Extract basic information
        website_info = info_extractor.extract_basic_info(url)

        # Step 2: Analyze security
        security_result = security_analyzer.analyze_security(url)

        # Step 3: Predict future risks
        risk_prediction = risk_predictor.predict_future_risk(security_result)

        # Combine results
        result = {
            'url': url,
            'scan_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'scan_type': scan_type,
            'website_info': website_info,
            'risk_level_text': security_result['risk_level_text'],
            'risk_score': security_result['risk_score'],
            'vulnerability_count': security_result['vulnerability_count'],
            'vulnerabilities': security_result['vulnerabilities'],
            'security_headers': security_result['security_headers'],
            'ssl_info': security_result['ssl_info'],
            'domain_security': security_result['domain_security'],
            'content_security': security_result['content_security'],
            'recommendations': security_result['recommendations'],
            'probability': risk_prediction['probability'],
            'future_risk': risk_prediction['overall_risk']
        }

        # Generate a unique ID for the result
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        result_id = f"{url.replace('://', '_').replace('/', '_').replace('.', '_')}_{timestamp}"

        # Save the result
        with open(f'reports/{result_id}.json', 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=4)

        # Return the response
        return jsonify({
            'status': 'success',
            'result_id': result_id,
            'summary': {
                'url': result['url'],
                'risk_level': result['risk_level_text'],
                'risk_score': result['risk_score'],
                'vulnerability_count': result['vulnerability_count']
            }
        })

    except Exception as e:
        print(f"Error during analysis: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/api/reports/<result_id>', methods=['GET'])
def get_report(result_id):
    try:
        with open(f'reports/{result_id}.json', 'r', encoding='utf-8') as f:
            report = json.load(f)
        return jsonify(report)
    except FileNotFoundError:
        return jsonify({'error': 'Report not found'}), 404

@app.route('/api/reports', methods=['GET'])
def list_reports():
    reports = []

    # Check if reports directory exists
    if not os.path.exists('reports'):
        return jsonify(reports)

    # List files in reports directory
    for filename in os.listdir('reports'):
        if filename.endswith('.json'):
            try:
                with open(f'reports/{filename}', 'r', encoding='utf-8') as f:
                    report = json.load(f)
                reports.append({
                    'id': filename.replace('.json', ''),
                    'url': report['url'],
                    'scan_date': report['scan_date'],
                    'risk_level': report['risk_level_text'],
                    'risk_score': report['risk_score']
                })
            except:
                # Skip invalid files
                continue

    # Sort by scan date (most recent first)
    reports.sort(key=lambda x: x['scan_date'], reverse=True)

    return jsonify(reports)

@app.route('/api/generate-report/<result_id>/<format>', methods=['GET'])
def generate_report(result_id, format):
    try:
        # Check if report exists
        report_path = f'reports/{result_id}.json'
        if not os.path.exists(report_path):
            return jsonify({'error': 'Report not found'}), 404

        # Load the report
        with open(report_path, 'r', encoding='utf-8') as f:
            report = json.load(f)

        # Generate report in requested format
        if format == 'json':
            # For JSON, we just return the existing file
            report_file = f'{result_id}.json'
        elif format == 'html':
            # Generate HTML report
            report_file = f'{result_id}.html'

            # Example HTML generation (simplistic)
            with open(f'reports/{report_file}', 'w', encoding='utf-8') as f:
                f.write(f"<html><body><h1>Report for {report['url']}</h1></body></html>")
        else:
            return jsonify({'error': 'Unsupported format'}), 400

        return jsonify({
            'status': 'success',
            'report_file': report_file
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/download-report/<filename>', methods=['GET'])
def download_report(filename):
    return send_from_directory('reports', filename)

def start_serveo():
    """Start serveo.net to expose the API publicly"""
    # Install ssh if not already installed
    os.system("apt-get install -y ssh")

    # Generate SSH key if not already present
    if not os.path.exists('/root/.ssh/id_rsa'):
        os.system("mkdir -p /root/.ssh")
        os.system("ssh-keygen -t rsa -N '' -f /root/.ssh/id_rsa")

    # Create a random subdomain
    import random
    import string
    subdomain = ''.join(random.choices(string.ascii_lowercase + string.digits, k=10))

    # Start serveo.net tunnel
    process = subprocess.Popen(
        f"ssh -o StrictHostKeyChecking=no -R websecure-{subdomain}:80:localhost:5000 serveo.net",
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    # Wait for serveo to start and print the URL
    for line in iter(process.stdout.readline, ''):
        if 'forwarding' in line.lower() and 'https://' in line:
            url = line.split('https://')[1].split()[0]
            url = f"https://{url}"
            print("="*70)
            print(f"Public URL: {url}")
            print(f"API URL: {url}/api")
            print("COPY THIS URL and use it in your frontend's api.js file")
            print("="*70)
            break

    return process

if __name__ == '__main__':
    # Start serveo.net in a separate thread
    tunnel_thread = threading.Thread(target=start_serveo)
    tunnel_thread.daemon = True
    tunnel_thread.start()

    # Wait for serveo to initialize
    time.sleep(5)

    # Run Flask app
    app.run(debug=True, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


Public URL: https://34ad67b98958938d845bd6550a5e532a.serveo.net
API URL: https://34ad67b98958938d845bd6550a5e532a.serveo.net/api
COPY THIS URL and use it in your frontend's api.js file


new verison of app.py

In [ ]:
!ls /content


app.py	__pycache__  risk_predictor.py	security_analyzer.py
models	reports      sample_data	website_info_extractor.py


In [ ]:
from website_info_extractor import WebsiteInfoExtractor
from security_analyzer import SecurityAnalyzer
from risk_predictor import RiskPredictor


In [ ]:
# Example test to check if the import works
website_info = WebsiteInfoExtractor()  # This should work if the import is correct


In [ ]:
%%writefile app.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import json
from datetime import datetime
import time
from pyngrok import ngrok

# Import custom modules
from website_info_extractor import WebsiteInfoExtractor
from security_analyzer import SecurityAnalyzer
from risk_predictor import RiskPredictor

app = Flask(__name__)
CORS(app, resources={r"/api/*": {"origins": "*"}})

# Create necessary directories
os.makedirs('reports', exist_ok=True)
os.makedirs('models', exist_ok=True)

# Initialize analyzers
info_extractor = WebsiteInfoExtractor()
security_analyzer = SecurityAnalyzer()
risk_predictor = RiskPredictor()

@app.route('/api/health', methods=['GET'])
def health_check():
    return jsonify({
        'status': 'healthy',
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'version': '1.0.0'
    })

@app.route('/api/analyze', methods=['POST'])
def analyze_website():
    data = request.get_json()

    if not data:
        return jsonify({'error': 'Invalid JSON data'}), 400

    url = data.get('url')
    scan_type = data.get('scan_type', 'quick')

    if not url:
        return jsonify({'error': 'URL is required'}), 400

    try:
        # Step 1: Extract basic information
        website_info = info_extractor.extract_basic_info(url)

        # Step 2: Analyze security
        security_result = security_analyzer.analyze_security(url)

        # Step 3: Predict future risks
        risk_prediction = risk_predictor.predict_future_risk(security_result)

        # Combine results
        result = {
            'url': url,
            'scan_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'scan_type': scan_type,
            'website_info': website_info,
            'risk_level_text': security_result['risk_level_text'],
            'risk_score': security_result['risk_score'],
            'vulnerability_count': security_result['vulnerability_count'],
            'vulnerabilities': security_result['vulnerabilities'],
            'security_headers': security_result['security_headers'],
            'ssl_info': security_result['ssl_info'],
            'domain_security': security_result['domain_security'],
            'content_security': security_result['content_security'],
            'recommendations': security_result['recommendations'],
            'probability': risk_prediction['probability'],
            'future_risk': risk_prediction['overall_risk']
        }

        # Generate a unique ID for the result
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        result_id = f"{url.replace('://', '_').replace('/', '_').replace('.', '_')}_{timestamp}"

        # Save the result
        with open(f'reports/{result_id}.json', 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=4)

        # Return the response
        return jsonify({
            'status': 'success',
            'result_id': result_id,
            'summary': {
                'url': result['url'],
                'risk_level': result['risk_level_text'],
                'risk_score': result['risk_score'],
                'vulnerability_count': result['vulnerability_count']
            }
        })

    except Exception as e:
        print(f"Error during analysis: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/api/reports/<result_id>', methods=['GET'])
def get_report(result_id):
    try:
        with open(f'reports/{result_id}.json', 'r', encoding='utf-8') as f:
            report = json.load(f)
        return jsonify(report)
    except FileNotFoundError:
        return jsonify({'error': 'Report not found'}), 404

@app.route('/api/reports', methods=['GET'])
def list_reports():
    reports = []

    # Check if reports directory exists
    if not os.path.exists('reports'):
        return jsonify(reports)

    # List files in reports directory
    for filename in os.listdir('reports'):
        if filename.endswith('.json'):
            try:
                with open(f'reports/{filename}', 'r', encoding='utf-8') as f:
                    report = json.load(f)
                reports.append({
                    'id': filename.replace('.json', ''),
                    'url': report['url'],
                    'scan_date': report['scan_date'],
                    'risk_level': report['risk_level_text'],
                    'risk_score': report['risk_score']
                })
            except:
                # Skip invalid files
                continue

    # Sort by scan date (most recent first)
    reports.sort(key=lambda x: x['scan_date'], reverse=True)

    return jsonify(reports)

@app.route('/api/generate-report/<result_id>/<format>', methods=['GET'])
def generate_report(result_id, format):
    try:
        # Check if report exists
        report_path = f'reports/{result_id}.json'
        if not os.path.exists(report_path):
            return jsonify({'error': 'Report not found'}), 404

        # Load the report
        with open(report_path, 'r', encoding='utf-8') as f:
            report = json.load(f)

        # Generate report in requested format
        if format == 'json':
            # For JSON, we just return the existing file
            report_file = f'{result_id}.json'
        elif format == 'html':
            # Generate HTML report
            html_content = generate_html_report(report)
            report_file = f'{result_id}.html'

            with open(f'reports/{report_file}', 'w', encoding='utf-8') as f:
                f.write(html_content)
        else:
            return jsonify({'error': 'Unsupported format'}), 400

        return jsonify({
            'status': 'success',
            'report_file': report_file
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/download-report/<filename>', methods=['GET'])
def download_report(filename):
    return send_from_directory('reports', filename)

# Function to generate HTML report
def generate_html_report(report):
    # Simple HTML report generation
    html = f'''
    <!DOCTYPE html>
    <html lang="fr">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Rapport de sécurité - {report['url']}</title>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; color: #333; max-width: 1200px; margin: 0 auto; padding: 20px; }}
            h1, h2, h3 {{ color: #2c3e50; }}
            .header {{ background-color: #f8f9fa; padding: 20px; border-radius: 5px; margin-bottom: 20px; }}
            .section {{ margin-bottom: 30px; }}
            .vuln-card {{ border: 1px solid #ddd; padding: 15px; margin-bottom: 10px; border-radius: 5px; }}
            .vuln-critical {{ border-left: 5px solid #dc3545; }}
            .vuln-important {{ border-left: 5px solid #fd7e14; }}
            .vuln-modere {{ border-left: 5px solid #ffc107; }}
            .vuln-faible {{ border-left: 5px solid #28a745; }}
            .metrics {{ display: flex; flex-wrap: wrap; gap: 20px; margin-bottom: 20px; }}
            .metric-card {{ flex: 1; min-width: 200px; background-color: #f8f9fa; padding: 15px; border-radius: 5px; text-align: center; }}
            .metric-value {{ font-size: 24px; font-weight: bold; margin: 10px 0; }}
            .recommendation {{ background-color: #e9ecef; padding: 15px; margin-bottom: 10px; border-radius: 5px; }}
            table {{ width: 100%; border-collapse: collapse; margin-bottom: 20px; }}
            th, td {{ padding: 10px; text-align: left; border-bottom: 1px solid #ddd; }}
            th {{ background-color: #f2f2f2; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1>Rapport de sécurité</h1>
            <p>URL: {report['url']}</p>
            <p>Date de l'analyse: {report['scan_date']}</p>
        </div>

        <div class="section">
            <h2>Résumé</h2>
            <div class="metrics">
                <div class="metric-card">
                    <h3>Niveau de risque</h3>
                    <div class="metric-value">{report['risk_level_text']}</div>
                </div>
                <div class="metric-card">
                    <h3>Score de sécurité</h3>
                    <div class="metric-value">{report['risk_score']}%</div>
                </div>
                <div class="metric-card">
                    <h3>Vulnérabilités</h3>
                    <div class="metric-value">{report['vulnerability_count']}</div>
                </div>
            </div>
        </div>

        <div class="section">
            <h2>Vulnérabilités détectées</h2>
    '''

    # Add vulnerabilities
    if report['vulnerabilities']:
        for vuln in report['vulnerabilities']:
            severity_class = f"vuln-{vuln['severity'].lower()}"
            html += f'''
            <div class="vuln-card {severity_class}">
                <h3>{vuln['name']}</h3>
                <p><strong>Sévérité:</strong> {vuln['severity']}</p>
                <p>{vuln['description']}</p>
            </div>
            '''
    else:
        html += '<p>Aucune vulnérabilité détectée.</p>'

    # Add recommendations
    html += '''
        </div>

        <div class="section">
            <h2>Recommandations</h2>
    '''

    if report['recommendations']:
        for rec in report['recommendations']:
            html += f'''
            <div class="recommendation">
                <h3>{rec['title']}</h3>
                <p><strong>Priorité:</strong> {rec['priority']}</p>
                <p>{rec['description']}</p>
            </div>
            '''
    else:
        html += '<p>Aucune recommandation spécifique.</p>'

    # Add technical details
    html += '''
        </div>

        <div class="section">
            <h2>Détails techniques</h2>

            <h3>En-têtes de sécurité HTTP</h3>
            <table>
                <tr>
                    <th>En-tête</th>
                    <th>Valeur</th>
                </tr>
    '''

    for header, value in report['security_headers'].items():
        if header != 'score':
            html += f'''
            <tr>
                <td>{header}</td>
                <td>{value if value else "Non défini"}</td>
            </tr>
            '''

    html += '''
            </table>

            <h3>SSL/TLS</h3>
    '''

    if report['ssl_info'].get('has_ssl', False):
        html += f'''
        <p>SSL/TLS activé: <span style="color: green;">Oui</span></p>
        <p>Certificat valide: <span style="color: {'green' if report['ssl_info'].get('certificate_valid', False) else 'red'};">{'Oui' if report['ssl_info'].get('certificate_valid', False) else 'Non'}</span></p>
        <p>Date d'expiration: {report['ssl_info'].get('certificate_expiry', 'Inconnue')}</p>
        <p>Version du protocole: {report['ssl_info'].get('protocol_version', 'Inconnue')}</p>
        '''
    else:
        html += '<p>SSL/TLS activé: <span style="color: red;">Non</span></p>'

    # End the HTML document
    html += '''
        </div>
    </body>
    </html>
    '''

    return html

def start_ngrok():
    """Start ngrok and make API publicly accessible"""
    try:
        # Set your ngrok auth token (if you have one)
        ngrok.set_auth_token("2wYJXTr6naFYAu83Shzevx604IE_m5GoMeYMCJSw3TNVhcgD")

        # Open HTTP tunnel on port 5000 (Flask's default port)
        http_tunnel = ngrok.connect(5000)

        # Get the public URL
        public_url = http_tunnel.public_url

        print("=" * 70)
        print(f"Public URL: {public_url}")
        print(f"API URL: {public_url}/api")
        print("COPY THIS URL. You need to update the API_BASE_URL in api.js!")
        print("=" * 70)

        return public_url
    except Exception as e:
        print(f"Error starting ngrok: {e}")
        return None

if __name__ == '__main__':
    # Start ngrok
    public_url = start_ngrok()

    # Run Flask app
    app.run(debug=True, host='0.0.0.0', port=5000)

Overwriting app.py


In [ ]:


%%writefile security_analyzer.py
# Coller ici le contenu de security_analyzer.py
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse
import ssl
import socket
from datetime import datetime
import random

class SecurityAnalyzer:
    def __init__(self):
        self.user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'

    def analyze_security(self, url):
        """Analyser la sécurité d'un site web"""
        result = {
            'url': url,
            'scan_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'risk_level_text': 'Modéré',  # Valeur par défaut
            'risk_score': 5.0,  # Valeur par défaut
            'vulnerabilities': [],
            'security_headers': {},
            'ssl_info': {},
            'domain_security': {},
            'content_security': {},
            'is_anomaly': False,
            'recommendations': []
        }

        try:
            # Normaliser l'URL
            if not url.startswith(('http://', 'https://')):
                url = 'https://' + url

            parsed_url = urlparse(url)
            domain = parsed_url.netloc

            # Obtenir le contenu HTML
            headers = {'User-Agent': self.user_agent}
            response = requests.get(url, headers=headers, timeout=10)

            # Analyser les en-têtes de sécurité
            result['security_headers'] = self._analyze_security_headers(response.headers)

            # Analyser la configuration SSL/TLS
            result['ssl_info'] = self._analyze_ssl(domain)

            # Analyser la sécurité du contenu
            result['content_security'] = self._analyze_content_security(response.content)

            # Analyser la sécurité du domaine
            result['domain_security'] = self._analyze_domain_security(domain)

            # Identifier les vulnérabilités
            self._identify_vulnerabilities(result)

            # Calculer le score de risque global
            self._calculate_risk_score(result)

            # Générer des recommandations
            self._generate_recommendations(result)

            return result

        except Exception as e:
            print(f"Erreur lors de l'analyse de sécurité: {e}")
            result['vulnerabilities'].append({
                'id': 'ERROR-01',
                'name': 'Erreur d\'analyse',
                'severity': 'Modéré',
                'description': f"Une erreur s'est produite lors de l'analyse: {str(e)}"
            })
            return result

    def _analyze_security_headers(self, headers):
        """Analyser les en-têtes de sécurité HTTP"""
        security_headers = {
            'Content-Security-Policy': headers.get('Content-Security-Policy', None),
            'X-XSS-Protection': headers.get('X-XSS-Protection', None),
            'X-Content-Type-Options': headers.get('X-Content-Type-Options', None),
            'X-Frame-Options': headers.get('X-Frame-Options', None),
            'Strict-Transport-Security': headers.get('Strict-Transport-Security', None),
            'Referrer-Policy': headers.get('Referrer-Policy', None),
            'Feature-Policy': headers.get('Feature-Policy', None),
            'Permissions-Policy': headers.get('Permissions-Policy', None)
        }

        # Ajouter un score pour les en-têtes
        security_headers['score'] = sum(1 for value in security_headers.values() if value is not None) / len(security_headers) * 10

        return security_headers

    def _analyze_ssl(self, domain):
        """Analyser la configuration SSL/TLS"""
        ssl_info = {
            'has_ssl': False,
            'certificate_valid': False,
            'certificate_expiry': None,
            'protocol_version': None,
            'score': 0
        }

        try:
            # Simuler une vérification SSL (pour le prototype)
            # Dans une version réelle, utilisez socket et ssl
            ssl_info['has_ssl'] = True
            ssl_info['certificate_valid'] = random.random() > 0.2  # 80% de chance d'être valide

            # Date d'expiration simulée
            expiry_year = datetime.now().year + 1
            expiry_month = random.choice(["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
            expiry_day = str(random.randint(1, 28)).zfill(2)
            ssl_info['certificate_expiry'] = f"{expiry_day} {expiry_month} {expiry_year}"

            # Version TLS simulée
            ssl_info['protocol_version'] = random.choice(["TLSv1.2", "TLSv1.3"])

            # Score basé sur la présence et la validité du certificat
            ssl_info['score'] = 10 if ssl_info['certificate_valid'] else 5

        except:
            ssl_info['score'] = 0

        return ssl_info

    def _analyze_content_security(self, content):
        """Analyser la sécurité du contenu HTML"""
        content_security = {
            'has_csrf_protection': False,
            'has_xss_protection': False,
            'external_scripts': 0,
            'form_security': False,
            'score': 0
        }

        try:
            soup = BeautifulSoup(content, 'html.parser')

            # Vérifier la protection CSRF
            csrf_tokens = soup.find_all(['input', 'meta'], attrs={'name': re.compile('csrf|_token', re.I)})
            content_security['has_csrf_protection'] = len(csrf_tokens) > 0

            # Compter les scripts externes
            scripts = soup.find_all('script', src=True)
            content_security['external_scripts'] = len(scripts)

            # Vérifier la sécurité des formulaires
            forms = soup.find_all('form')
            secure_forms = 0
            for form in forms:
                if form.get('method', '').lower() == 'post' and any(token in str(form).lower() for token in ['csrf', 'token']):
                    secure_forms += 1

            content_security['form_security'] = secure_forms == len(forms) and len(forms) > 0

            # Calculer le score
            score = 0
            if content_security['has_csrf_protection']:
                score += 3
            if content_security['form_security']:
                score += 4
            if content_security['external_scripts'] < 5:
                score += 3

            content_security['score'] = score

        except Exception as e:
            print(f"Erreur lors de l'analyse du contenu: {e}")
            content_security['score'] = 0

        return content_security

    def _analyze_domain_security(self, domain):
        """Analyser la sécurité du domaine"""
        domain_security = {
            'has_spf': False,
            'has_dkim': False,
            'has_dmarc': False,
            'has_caa': False,
            'score': 0
        }

        try:
            # Simulation pour le prototype
            # Dans une version réelle, utilisez dns.resolver
            domain_security['has_spf'] = random.random() > 0.3
            domain_security['has_dmarc'] = random.random() > 0.5
            domain_security['has_caa'] = random.random() > 0.7

            # Calculer le score
            score = 0
            if domain_security['has_spf']:
                score += 3
            if domain_security['has_dmarc']:
                score += 3
            if domain_security['has_caa']:
                score += 4

            domain_security['score'] = score

        except Exception as e:
            print(f"Erreur lors de l'analyse du domaine: {e}")
            domain_security['score'] = 0

        return domain_security

    def _identify_vulnerabilities(self, result):
        """Identifier les vulnérabilités potentielles"""
        vulnerabilities = []

        # Vulnérabilités liées aux en-têtes de sécurité
        if not result['security_headers'].get('Content-Security-Policy'):
            vulnerabilities.append({
                'id': 'CSP-01',
                'name': 'Absence de Content-Security-Policy',
                'severity': 'Important',
                'description': 'La politique de sécurité du contenu (CSP) est absente, ce qui augmente le risque d\'attaques XSS.'
            })

        if not result['security_headers'].get('X-XSS-Protection'):
            vulnerabilities.append({
                'id': 'XSS-01',
                'name': 'Protection XSS non activée',
                'severity': 'Modéré',
                'description': 'L\'en-tête X-XSS-Protection n\'est pas défini, ce qui réduit la protection contre les attaques XSS.'
            })

        if not result['security_headers'].get('X-Frame-Options'):
            vulnerabilities.append({
                'id': 'FRAME-01',
                'name': 'Clickjacking possible',
                'severity': 'Modéré',
                'description': 'L\'en-tête X-Frame-Options n\'est pas défini, ce qui rend le site vulnérable au clickjacking.'
            })

        if not result['security_headers'].get('Strict-Transport-Security'):
            vulnerabilities.append({
                'id': 'HSTS-01',
                'name': 'HSTS non configuré',
                'severity': 'Modéré',
                'description': 'L\'en-tête Strict-Transport-Security n\'est pas défini, ce qui pourrait permettre des attaques de type downgrade.'
            })

        # Vulnérabilités liées à SSL/TLS
        if not result['ssl_info'].get('has_ssl', False):
            vulnerabilities.append({
                'id': 'SSL-01',
                'name': 'SSL/TLS non utilisé',
                'severity': 'Critique',
                'description': 'Le site n\'utilise pas SSL/TLS, ce qui rend toutes les communications vulnérables à l\'interception.'
            })

        # Vulnérabilités liées au contenu
        if not result['content_security'].get('has_csrf_protection', False):
            vulnerabilities.append({
                'id': 'CSRF-01',
                'name': 'Protection CSRF absente',
                'severity': 'Important',
                'description': 'Aucune protection CSRF détectée, ce qui pourrait permettre des attaques par falsification de requête.'
            })

# Vulnérabilités liées au domaine (suite)
        if not result['domain_security'].get('has_dmarc', False):
            vulnerabilities.append({
                'id': 'DMARC-01',
                'name': 'Enregistrement DMARC absent',
                'severity': 'Modéré',
                'description': 'Aucun enregistrement DMARC détecté, ce qui réduit la protection contre l\'usurpation d\'identité par e-mail.'
            })

        # Ajouter d'autres vulnérabilités simulées pour une meilleure expérience utilisateur
        # Ces vulnérabilités seraient normalement détectées par des analyses plus poussées
        if random.random() > 0.7:  # 30% de chance
            vulnerabilities.append({
                'id': 'INJ-01',
                'name': 'Points d\'injection SQL potentiels',
                'severity': 'Critique',
                'description': 'Des paramètres de requête non sécurisés ont été détectés, pouvant conduire à des injections SQL.'
            })

        if random.random() > 0.8:  # 20% de chance
            vulnerabilities.append({
                'id': 'AUTH-01',
                'name': 'Faiblesses dans l\'authentification',
                'severity': 'Important',
                'description': 'Le site n\'implémente pas de protection contre les attaques par force brute sur les formulaires d\'authentification.'
            })

        result['vulnerabilities'] = vulnerabilities
        result['vulnerability_count'] = len(vulnerabilities)

    def _calculate_risk_score(self, result):
        """Calculer le score de risque global"""
        # Pondération des différentes catégories
        weights = {
            'security_headers': 0.25,
            'ssl_info': 0.30,
            'content_security': 0.25,
            'domain_security': 0.20
        }

        # Calculer le score pondéré
        weighted_score = (
            result['security_headers'].get('score', 0) * weights['security_headers'] +
            result['ssl_info'].get('score', 0) * weights['ssl_info'] +
            result['content_security'].get('score', 0) * weights['content_security'] +
            result['domain_security'].get('score', 0) * weights['domain_security']
        )

        # Ajuster le score en fonction des vulnérabilités
        vulnerability_penalty = 0
        for vuln in result['vulnerabilities']:
            if vuln['severity'] == 'Critique':
                vulnerability_penalty += 2.0
            elif vuln['severity'] == 'Important':
                vulnerability_penalty += 1.0
            elif vuln['severity'] == 'Modéré':
                vulnerability_penalty += 0.5

        # Score final (sur 10)
        final_score = max(0, weighted_score - vulnerability_penalty)

        # Convertir en pourcentage
        result['risk_score'] = round(final_score * 10, 1)

        # Définir le niveau de risque textuel
        if result['risk_score'] >= 80:
            result['risk_level_text'] = 'Faible'
        elif result['risk_score'] >= 60:
            result['risk_level_text'] = 'Modéré'
        elif result['risk_score'] >= 40:
            result['risk_level_text'] = 'Important'
        else:
            result['risk_level_text'] = 'Critique'

    def _generate_recommendations(self, result):
        """Générer des recommandations basées sur les vulnérabilités"""
        recommendations = []

        # Recommandations pour les en-têtes de sécurité
        if not result['security_headers'].get('Content-Security-Policy'):
            recommendations.append({
                'title': 'Mettre en place une politique de sécurité du contenu (CSP)',
                'description': 'Implémentez l\'en-tête Content-Security-Policy pour réduire le risque d\'attaques XSS.',
                'priority': 'Haute'
            })

        if not result['security_headers'].get('X-XSS-Protection'):
            recommendations.append({
                'title': 'Activer la protection XSS du navigateur',
                'description': 'Ajoutez l\'en-tête X-XSS-Protection pour activer la protection XSS intégrée au navigateur.',
                'priority': 'Moyenne'
            })

        if not result['security_headers'].get('X-Frame-Options'):
            recommendations.append({
                'title': 'Prévenir le clickjacking',
                'description': 'Ajoutez l\'en-tête X-Frame-Options pour empêcher votre site d\'être chargé dans un iframe.',
                'priority': 'Moyenne'
            })

        if not result['security_headers'].get('Strict-Transport-Security'):
            recommendations.append({
                'title': 'Activer HSTS',
                'description': 'Implémentez l\'en-tête Strict-Transport-Security pour forcer les connexions HTTPS.',
                'priority': 'Haute'
            })

        # Recommandations pour SSL/TLS
        if not result['ssl_info'].get('has_ssl', False):
            recommendations.append({
                'title': 'Activer SSL/TLS',
                'description': 'Mettez en place SSL/TLS pour chiffrer toutes les communications avec votre site.',
                'priority': 'Critique'
            })

        # Recommandations pour le contenu
        if not result['content_security'].get('has_csrf_protection', False):
            recommendations.append({
                'title': 'Ajouter une protection CSRF',
                'description': 'Implémentez des jetons CSRF dans tous les formulaires pour prévenir les attaques par falsification de requête.',
                'priority': 'Haute'
            })

        # Recommandations pour le domaine
        if not result['domain_security'].get('has_spf', False):
            recommendations.append({
                'title': 'Configurer un enregistrement SPF',
                'description': 'Ajoutez un enregistrement SPF pour prévenir l\'usurpation d\'identité par e-mail.',
                'priority': 'Moyenne'
            })

        if not result['domain_security'].get('has_dmarc', False):
            recommendations.append({
                'title': 'Configurer un enregistrement DMARC',
                'description': 'Ajoutez un enregistrement DMARC pour améliorer la protection contre l\'usurpation d\'identité par e-mail.',
                'priority': 'Moyenne'
            })

        # Recommandations pour les autres vulnérabilités
        for vuln in result['vulnerabilities']:
            if vuln['id'] == 'INJ-01':
                recommendations.append({
                    'title': 'Sécuriser les entrées utilisateur contre les injections SQL',
                    'description': 'Utilisez des requêtes paramétrées ou des ORM pour toutes les interactions avec la base de données.',
                    'priority': 'Critique'
                })
            elif vuln['id'] == 'AUTH-01':
                recommendations.append({
                    'title': 'Renforcer le système d\'authentification',
                    'description': 'Implémentez des protections contre les attaques par force brute comme le verrouillage de compte ou le CAPTCHA.',
                    'priority': 'Haute'
                })

        result['recommendations'] = recommendations

Overwriting security_analyzer.py


In [ ]:
%%writefile website_info_extractor.py
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import re
import time

class WebsiteInfoExtractor:
    def __init__(self):
        self.user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        self.timeout = 10

    def extract_basic_info(self, url):
        """Extraire les informations de base d'un site web"""
        # Initialiser le dictionnaire résultat
        info = {
            'url': url,
            'name': None,
            'title': None,
            'description': None,
            'logo_url': None,
            'favicon_url': None,
            'server_info': None,
            'technologies': [],
            'domain_info': {
                'registrar': None,
                'registration_date': None,
                'expiry_date': None
            }
        }

        try:
            # Normaliser l'URL
            if not url.startswith(('http://', 'https://')):
                url = 'https://' + url

            # Extraire le domaine
            parsed_url = urlparse(url)
            domain = parsed_url.netloc

            # Récupérer le contenu HTML
            headers = {'User-Agent': self.user_agent}
            response = requests.get(url, headers=headers, timeout=self.timeout)

            # Extraire les informations du serveur depuis les en-têtes HTTP
            info['server_info'] = response.headers.get('Server')

            # Analyser le HTML
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraire le titre
            title_tag = soup.find('title')
            if title_tag:
                info['title'] = title_tag.text.strip()
                # Utiliser le titre comme nom par défaut si aucun nom n'est trouvé
                info['name'] = info['title']

            # Extraire la description
            meta_desc = soup.find('meta', attrs={'name': 'description'})
            if meta_desc:
                info['description'] = meta_desc.get('content', '').strip()

            # Rechercher le logo (plusieurs stratégies)
            # 1. Chercher dans les liens avec "logo" dans l'URL ou la classe
            logo_links = soup.find_all('a', href=lambda href: href and 'logo' in href.lower())
            logo_imgs = soup.find_all('img', src=lambda src: src and ('logo' in src.lower() or 'brand' in src.lower()))
            logo_divs = soup.find_all(['div', 'span'], class_=lambda cls: cls and ('logo' in cls.lower() or 'brand' in cls.lower()))

            # 2. Chercher dans le header/navbar
            header = soup.find(['header', 'nav'])
            header_logos = []
            if header:
                header_logos = header.find_all('img')

            # Combinaison des résultats
            potential_logos = []
            for img in logo_imgs:
                potential_logos.append(img.get('src'))

            for div in logo_divs:
                img = div.find('img')
                if img and img.get('src'):
                    potential_logos.append(img.get('src'))

            for link in logo_links:
                img = link.find('img')
                if img and img.get('src'):
                    potential_logos.append(img.get('src'))

            for img in header_logos:
                if img.get('src'):
                    potential_logos.append(img.get('src'))

            # Sélectionner le logo le plus probable
            if potential_logos:
                # Convertir en URL absolue si nécessaire
                logo_url = potential_logos[0]
                if not (logo_url.startswith('http://') or logo_url.startswith('https://')):
                    if logo_url.startswith('//'):
                        logo_url = 'https:' + logo_url
                    elif logo_url.startswith('/'):
                        logo_url = f"{parsed_url.scheme}://{domain}{logo_url}"
                    else:
                        logo_url = f"{parsed_url.scheme}://{domain}/{logo_url}"

                info['logo_url'] = logo_url

            # Extraire le favicon
            favicon_link = soup.find('link', rel=lambda rel: rel and ('icon' in rel.lower() or 'shortcut' in rel.lower()))
            if favicon_link and favicon_link.get('href'):
                favicon_url = favicon_link.get('href')
                if not (favicon_url.startswith('http://') or favicon_url.startswith('https://')):
                    if favicon_url.startswith('//'):
                        favicon_url = 'https:' + favicon_url
                    elif favicon_url.startswith('/'):
                        favicon_url = f"{parsed_url.scheme}://{domain}{favicon_url}"
                    else:
                        favicon_url = f"{parsed_url.scheme}://{domain}/{favicon_url}"

                info['favicon_url'] = favicon_url
            else:
                # URL favicon par défaut
                info['favicon_url'] = f"{parsed_url.scheme}://{domain}/favicon.ico"

            # Détecter les technologies utilisées (version simplifiée)
            technologies = []

            # HTML5
            if soup.find('html', attrs={'doctype': re.compile('html', re.I)}) or soup.find('html', attrs={'lang': True}):
                technologies.append('HTML5')

            # CSS
            if soup.find_all('link', rel='stylesheet') or soup.find_all('style'):
                technologies.append('CSS3')

            # JavaScript frameworks
            js_libs = {
                'jQuery': ['jquery'],
                'React': ['react', 'reactjs'],
                'Vue.js': ['vue', 'vuejs'],
                'Angular': ['angular', 'ng-'],
                'Bootstrap': ['bootstrap'],
                'Tailwind CSS': ['tailwind'],
                'Font Awesome': ['fontawesome', 'fa-'],
                'WordPress': ['wp-content', 'wp-includes', 'wordpress'],
                'Shopify': ['shopify'],
                'Wix': ['wix'],
                'Drupal': ['drupal'],
                'Joomla': ['joomla']
            }

            page_text = str(soup)
            for lib, keywords in js_libs.items():
                for keyword in keywords:
                    if keyword.lower() in page_text.lower():
                        technologies.append(lib)
                        break

            # Analyser les scripts
            scripts = soup.find_all('script', src=True)
            for script in scripts:
                script_src = script.get('src', '').lower()
                for lib, keywords in js_libs.items():
                    for keyword in keywords:
                        if keyword.lower() in script_src:
                            if lib not in technologies:
                                technologies.append(lib)

            # Ajouter les technologies détectées
            info['technologies'] = list(set(technologies))  # Éliminer les doublons

            # Simuler les informations de domaine (dans une version réelle, utilisez WHOIS)
            # Ces données sont fictives pour le prototype
            import datetime
            import random

            current_year = datetime.datetime.now().year
            registration_year = random.randint(current_year - 10, current_year - 1)
            expiry_year = random.randint(current_year, current_year + 5)

            months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
            registration_month = random.choice(months)
            expiry_month = random.choice(months)

            registration_day = str(random.randint(1, 28)).zfill(2)
            expiry_day = str(random.randint(1, 28)).zfill(2)

            info['domain_info']['registrar'] = random.choice([
                "GoDaddy.com, LLC",
                "Namecheap, Inc.",
                "OVH",
                "Amazon Registrar, Inc.",
                "Google LLC",
                "NameSilo, LLC"
            ])

            info['domain_info']['registration_date'] = f"{registration_day} {registration_month} {registration_year}"
            info['domain_info']['expiry_date'] = f"{expiry_day} {expiry_month} {expiry_year}"

            return info

        except Exception as e:
            print(f"Erreur lors de l'extraction des informations: {str(e)}")
            return info

Writing website_info_extractor.py


In [ ]:


%%writefile risk_predictor.py
import numpy as np
import random
from datetime import datetime

class RiskPredictor:
    def __init__(self):
        # Modèle simple basé sur des règles
        self.risk_factors = {
            'missing_security_headers': 0.3,
            'missing_ssl': 0.5,
            'weak_csrf_protection': 0.3,
            'missing_domain_security': 0.2,
            'vulnerability_count_weight': 0.1
        }

    def predict_future_risk(self, security_result):
        """Prédire le risque futur d'attaques"""
        prediction = {
            'overall_risk': 0,
            'probability': {
                'Faible': 0.0,
                'Modéré': 0.0,
                'Important': 0.0,
                'Critique': 0.0
            },
            'time_horizon': '6 mois',
            'risk_factors': [],
            'improvement_potential': 0.0
        }

        # Facteurs de risque
        risk_factors = []
        total_risk_score = 0

        # Évaluer les en-têtes de sécurité
        security_headers_score = security_result['security_headers'].get('score', 0) / 10
        if security_headers_score < 0.6:
            risk_factor = {
                'name': 'En-têtes de sécurité insuffisants',
                'impact': (1 - security_headers_score) * self.risk_factors['missing_security_headers'],
                'description': 'Les en-têtes HTTP de sécurité ne sont pas correctement configurés.'
            }
            risk_factors.append(risk_factor)
            total_risk_score += risk_factor['impact']

        # Évaluer SSL/TLS
        ssl_score = security_result['ssl_info'].get('score', 0) / 10
        if ssl_score < 0.8:
            risk_factor = {
                'name': 'Configuration SSL/TLS faible',
                'impact': (1 - ssl_score) * self.risk_factors['missing_ssl'],
                'description': 'La configuration SSL/TLS présente des faiblesses.'
            }
            risk_factors.append(risk_factor)
            total_risk_score += risk_factor['impact']

        # Évaluer la protection CSRF
        if not security_result['content_security'].get('has_csrf_protection', False):
            risk_factor = {
                'name': 'Protection CSRF insuffisante',
                'impact': self.risk_factors['weak_csrf_protection'],
                'description': 'Le site manque de protection contre les attaques CSRF.'
            }
            risk_factors.append(risk_factor)
            total_risk_score += risk_factor['impact']

        # Évaluer la sécurité du domaine
        domain_security_score = security_result['domain_security'].get('score', 0) / 10
        if domain_security_score < 0.5:
            risk_factor = {
                'name': 'Sécurité du domaine insuffisante',
                'impact': (1 - domain_security_score) * self.risk_factors['missing_domain_security'],
                'description': 'Les configurations de sécurité du domaine (SPF, DMARC) sont insuffisantes.'
            }
            risk_factors.append(risk_factor)
            total_risk_score += risk_factor['impact']

        # Impact du nombre de vulnérabilités
        vulnerability_count = security_result.get('vulnerability_count', 0)
        if vulnerability_count > 0:
            critical_count = sum(1 for v in security_result.get('vulnerabilities', []) if v['severity'] == 'Critique')
            high_count = sum(1 for v in security_result.get('vulnerabilities', []) if v['severity'] == 'Important')

            vuln_impact = (critical_count * 0.2 + high_count * 0.1 + (vulnerability_count - critical_count - high_count) * 0.05) * self.risk_factors['vulnerability_count_weight']

            risk_factor = {
                'name': f'Présence de {vulnerability_count} vulnérabilités',
                'impact': vuln_impact,
                'description': f'Le site présente {critical_count} vulnérabilités critiques et {high_count} importantes.'
            }
            risk_factors.append(risk_factor)
            total_risk_score += risk_factor['impact']

        # Normaliser le score total (0-1)
        normalized_score = min(1.0, total_risk_score)

        # Calculer les probabilités
        if normalized_score < 0.3:
            prediction['probability'] = {
                'Faible': 0.7,
                'Modéré': 0.2,
                'Important': 0.1,
                'Critique': 0.0
            }
        elif normalized_score < 0.5:
            prediction['probability'] = {
                'Faible': 0.4,
                'Modéré': 0.4,
                'Important': 0.2,
                'Critique': 0.0
            }
        elif normalized_score < 0.7:
            prediction['probability'] = {
                'Faible': 0.2,
                'Modéré': 0.3,
                'Important': 0.4,
                'Critique': 0.1
            }
        else:
            prediction['probability'] = {
                'Faible': 0.1,
                'Modéré': 0.2,
                'Important': 0.3,
                'Critique': 0.4
            }

        # Calculer le risque global (0-10)
        prediction['overall_risk'] = normalized_score * 10

        # Potentiel d'amélioration basé sur le nombre de recommandations
        recommendation_count = len(security_result.get('recommendations', []))
        if recommendation_count > 0:
            prediction['improvement_potential'] = min(0.8, recommendation_count * 0.1)

        return prediction

Overwriting risk_predictor.py


In [ ]:
%run app.py

Public URL: https://c2f9-34-82-193-159.ngrok-free.app
API URL: https://c2f9-34-82-193-159.ngrok-free.app/api
COPY THIS URL. You need to update the API_BASE_URL in api.js!
 * Serving Flask app 'app'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
import json
import os

# Créer le dossier reports s'il n'existe pas
os.makedirs('reports', exist_ok=True)

# Rapport de test
test_report = {
  "url": "https://example.com",
  "scan_date": "2025-05-02 20:15:30",
  "scan_type": "deep",
  "risk_level_text": "Modéré",
  "risk_score": 68.5,
  "vulnerability_count": 4,
  "is_anomaly": False,
  "vulnerabilities": [
    {
      "id": "CSP-01",
      "name": "Absence de Content-Security-Policy",
      "severity": "Important",
      "description": "La politique de sécurité du contenu (CSP) est absente, ce qui augmente le risque d'attaques XSS."
    },
    {
      "id": "HSTS-01",
      "name": "HSTS non configuré",
      "severity": "Modéré",
      "description": "L'en-tête Strict-Transport-Security n'est pas défini, ce qui pourrait permettre des attaques de type downgrade."
    },
    {
      "id": "FRAME-01",
      "name": "Clickjacking possible",
      "severity": "Modéré",
      "description": "L'en-tête X-Frame-Options n'est pas défini, ce qui rend le site vulnérable au clickjacking."
    },
    {
      "id": "CSRF-01",
      "name": "Protection CSRF absente",
      "severity": "Important",
      "description": "Aucune protection CSRF détectée, ce qui pourrait permettre des attaques par falsification de requête."
    }
  ],
  "recommendations": [
    {
      "title": "Mettre en place une politique de sécurité du contenu (CSP)",
      "description": "Implémentez l'en-tête Content-Security-Policy pour réduire le risque d'attaques XSS.",
      "priority": "Haute"
    },
    {
      "title": "Activer HSTS",
      "description": "Implémentez l'en-tête Strict-Transport-Security pour forcer les connexions HTTPS.",
      "priority": "Moyenne"
    },
    {
      "title": "Prévenir le clickjacking",
      "description": "Ajoutez l'en-tête X-Frame-Options pour empêcher votre site d'être chargé dans un iframe.",
      "priority": "Moyenne"
    },
    {
      "title": "Ajouter une protection CSRF",
      "description": "Implémentez des jetons CSRF dans tous les formulaires pour prévenir les attaques par falsification de requête.",
      "priority": "Haute"
    }
  ],
  "security_headers": {
    "Content-Security-Policy": None,
    "X-XSS-Protection": "1; mode=block",
    "X-Content-Type-Options": "nosniff",
    "X-Frame-Options": None,
    "Strict-Transport-Security": None,
    "Referrer-Policy": "no-referrer-when-downgrade",
    "Feature-Policy": None,
    "Permissions-Policy": None,
    "score": 4.0
  },
  "ssl_info": {
    "has_ssl": True,
    "certificate_valid": True,
    "certificate_expiry": "15 Oct 2025",
    "protocol_version": "TLSv1.3",
    "score": 10.0
  },
  "content_security": {
    "has_csrf_protection": False,
    "has_xss_protection": True,
    "external_scripts": 3,
    "form_security": False,
    "score": 3.0
  },
  "domain_security": {
    "has_spf": True,
    "has_dkim": False,
    "has_dmarc": False,
    "has_caa": True,
    "score": 7.0
  },
  "probability": {
    "Faible": 0.2,
    "Modéré": 0.5,
    "Important": 0.2,
    "Critique": 0.1
  },
  "future_risk": 5.4,
  "website_info": {
    "url": "https://example.com",
    "name": "Example Domain",
    "title": "Example Domain",
    "description": "This domain is for use in illustrative examples in documents.",
    "logo_url": "https://example.com/images/logo.png",
    "favicon_url": "https://example.com/favicon.ico",
    "server_info": "ECS (dcb/7EA5)",
    "technologies": ["HTML5", "CSS3", "TLSv1.3"]
  }
}


# Enregistrer le rapport de test
report_id = "example_com_test_report"
with open(f'reports/{report_id}.json', 'w', encoding='utf-8') as f:
    json.dump(test_report, f, ensure_ascii=False, indent=4)

print(f"Test report created: {report_id}")
print(f"Available reports: {os.listdir('reports')}")

Test report created: example_com_test_report
Available reports: ['example_com_test_report.json']


In [ ]:


# Configurer ngrok (remplacez par votre token)
ngrok_auth_token = "2wYJXTr6naFYAu83Shzevx604IE_m5GoMeYMCJSw3TNVhcgD"
ngrok.set_auth_token(ngrok_auth_token)

# Ouvrir le tunnel ngrok
http_tunnel = ngrok.connect(5000)
print("\n" + "="*70)
print(f"URL ngrok: {http_tunnel.public_url}")
print("COPIEZ CETTE URL. C'est l'adresse à utiliser dans votre frontend!")
print("="*70 + "\n")



# Fonction pour lancer Flask dans un thread séparé
def run_flask():
    subprocess.run(["python", "app.py"])

# Lancer Flask dans un thread séparé
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Garder la cellule en cours d'exécution
try:
    while True:
        time.sleep(60)
        print(".", end="", flush=True)
except KeyboardInterrupt:
    print("\nArrêt du serveur...")
    ngrok.kill()


URL ngrok: https://e5ef-34-82-193-159.ngrok-free.app
COPIEZ CETTE URL. C'est l'adresse à utiliser dans votre frontend!

.

..........

.....

..................
Arrêt du serveur...


collab uilt in feature API

In [6]:
# Import necessary libraries
!pip install flask flask-cors requests beautifulsoup4

from flask import Flask, request, jsonify
from flask_cors import CORS
import os
import json
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import random

# Create the Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Create reports directory
os.makedirs('reports', exist_ok=True)

# Simple security analyzer function
def analyze_website(url):
    try:
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url

        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
        response = requests.get(url, headers=headers, timeout=10)

        # Get basic info
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.find('title')
        title_text = title.text if title else url

        # Check security headers
        security_headers = {
            'Content-Security-Policy': response.headers.get('Content-Security-Policy', None),
            'X-XSS-Protection': response.headers.get('X-XSS-Protection', None),
            'X-Frame-Options': response.headers.get('X-Frame-Options', None),
            'Strict-Transport-Security': response.headers.get('Strict-Transport-Security', None)
        }

        # Generate risk score and vulnerabilities
        missing_headers = sum(1 for _, v in security_headers.items() if not v)
        risk_score = max(0, 100 - (missing_headers * 20))

        if risk_score >= 80:
            risk_level = 'Faible'
        elif risk_score >= 60:
            risk_level = 'Modéré'
        elif risk_score >= 40:
            risk_level = 'Important'
        else:
            risk_level = 'Critique'

        # Create vulnerabilities list
        vulnerabilities = []
        if not security_headers['Content-Security-Policy']:
            vulnerabilities.append({
                'id': 'CSP-01',
                'name': 'Content Security Policy manquante',
                'severity': 'Important',
                'description': 'La politique de sécurité du contenu est absente, ce qui augmente le risque d\'attaques XSS.'
            })

        if not security_headers['X-XSS-Protection']:
            vulnerabilities.append({
                'id': 'XSS-01',
                'name': 'Protection XSS non activée',
                'severity': 'Modéré',
                'description': 'L\'en-tête X-XSS-Protection n\'est pas défini.'
            })

        # Add some random vulnerabilities for demo purposes
        if random.random() > 0.7:
            vulnerabilities.append({
                'id': 'INJ-01',
                'name': 'Vulnérabilité d\'injection potentielle',
                'severity': 'Critique',
                'description': 'Des points d\'injection potentiels ont été détectés.'
            })

        # Recommendations based on vulnerabilities
        recommendations = []
        for vuln in vulnerabilities:
            recommendations.append({
                'title': f'Corriger {vuln["name"]}',
                'description': f'Résoudre le problème: {vuln["description"]}',
                'priority': 'Haute' if vuln['severity'] == 'Critique' else 'Moyenne'
            })

        # Result object
        result = {
            'url': url,
            'scan_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'website_info': {
                'title': title_text,
                'name': title_text,
                'description': 'Site web analysé par WebSecure'
            },
            'risk_level_text': risk_level,
            'risk_score': risk_score,
            'vulnerability_count': len(vulnerabilities),
            'vulnerabilities': vulnerabilities,
            'security_headers': security_headers,
            'recommendations': recommendations
        }

        return result

    except Exception as e:
        print(f"Error analyzing website: {e}")
        return {
            'url': url,
            'scan_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'risk_level_text': 'Erreur',
            'risk_score': 0,
            'vulnerability_count': 1,
            'vulnerabilities': [{
                'id': 'ERROR-01',
                'name': 'Erreur d\'analyse',
                'severity': 'Critique',
                'description': f"Une erreur s'est produite lors de l'analyse: {str(e)}"
            }]
        }

# API Routes
@app.route('/api/health', methods=['GET'])
def health_check():
    return jsonify({
        'status': 'healthy',
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    })

@app.route('/api/analyze', methods=['POST'])
def analyze_website_route():
    data = request.get_json()

    if not data or not data.get('url'):
        return jsonify({'error': 'URL is required'}), 400

    try:
        # Analyze the website
        result = analyze_website(data['url'])

        # Generate a unique ID for the result
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        result_id = f"{data['url'].replace('://', '_').replace('/', '_').replace('.', '_')}_{timestamp}"

        # Save the result
        with open(f'reports/{result_id}.json', 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=4)

        # Return the response
        return jsonify({
            'status': 'success',
            'result_id': result_id,
            'summary': {
                'url': result['url'],
                'risk_level': result['risk_level_text'],
                'risk_score': result['risk_score'],
                'vulnerability_count': result['vulnerability_count']
            }
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/reports/<result_id>', methods=['GET'])
def get_report(result_id):
    try:
        with open(f'reports/{result_id}.json', 'r', encoding='utf-8') as f:
            report = json.load(f)
        return jsonify(report)
    except FileNotFoundError:
        return jsonify({'error': 'Report not found'}), 404

@app.route('/api/reports', methods=['GET'])
def list_reports():
    reports = []

    for filename in os.listdir('reports'):
        if filename.endswith('.json'):
            try:
                with open(f'reports/{filename}', 'r', encoding='utf-8') as f:
                    report = json.load(f)
                reports.append({
                    'id': filename.replace('.json', ''),
                    'url': report['url'],
                    'scan_date': report['scan_date'],
                    'risk_level': report['risk_level_text'],
                    'risk_score': report['risk_score']
                })
            except:
                continue

    # Sort by scan date (most recent first)
    reports.sort(key=lambda x: x['scan_date'], reverse=True)

    return jsonify(reports)

In [7]:
# Run the Flask app using Google Colab's public URL feature
from google.colab import output
print("Starting server...")

# Start the Flask app
from threading import Thread
def run_flask():
    app.run(host='0.0.0.0', port=5000)

flask_thread = Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Get the public URL
print("Getting public URL...")
public_url = output.eval_js('google.colab.kernel.proxyPort(5000)')
print(f"Your public API URL is: {public_url}/api")
print(f"Update your api.js file with this URL!")
print("API_BASE_URL = '" + public_url + "/api'")

# Keep the notebook running
import time
print("Server is running. Keep this tab open. Press Ctrl+C to stop.")
while True:
    time.sleep(60)
    print(".", end="", flush=True)


Starting server...
Getting public URL...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Your public API URL is: https://5000-m-s-203y6hpwtryt7-c.us-west1-0.prod.colab.dev/api
Update your api.js file with this URL!
API_BASE_URL = 'https://5000-m-s-203y6hpwtryt7-c.us-west1-0.prod.colab.dev/api'
Server is running. Keep this tab open. Press Ctrl+C to stop.
..

KeyboardInterrupt: 

In [8]:
# Import necessary libraries
!pip install flask flask-cors requests beautifulsoup4

from flask import Flask, request, jsonify, make_response
from flask_cors import CORS
import os
import json
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import random

# Create the Flask app
app = Flask(__name__)

# Configure CORS properly - this is the key fix
CORS(app, resources={r"/*": {"origins": "*"}}, supports_credentials=True)

# Additional CORS handling for all routes
@app.after_request
def after_request(response):
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
    response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE,OPTIONS')
    return response

# Create reports directory
os.makedirs('reports', exist_ok=True)

# Simple security analyzer function
def analyze_website(url):
    try:
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url

        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
        response = requests.get(url, headers=headers, timeout=10)

        # Get basic info
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.find('title')
        title_text = title.text if title else url

        # Check security headers
        security_headers = {
            'Content-Security-Policy': response.headers.get('Content-Security-Policy', None),
            'X-XSS-Protection': response.headers.get('X-XSS-Protection', None),
            'X-Frame-Options': response.headers.get('X-Frame-Options', None),
            'Strict-Transport-Security': response.headers.get('Strict-Transport-Security', None)
        }

        # Generate risk score and vulnerabilities
        missing_headers = sum(1 for _, v in security_headers.items() if not v)
        risk_score = max(0, 100 - (missing_headers * 20))

        if risk_score >= 80:
            risk_level = 'Faible'
        elif risk_score >= 60:
            risk_level = 'Modéré'
        elif risk_score >= 40:
            risk_level = 'Important'
        else:
            risk_level = 'Critique'

        # Create vulnerabilities list
        vulnerabilities = []
        if not security_headers['Content-Security-Policy']:
            vulnerabilities.append({
                'id': 'CSP-01',
                'name': 'Content Security Policy manquante',
                'severity': 'Important',
                'description': 'La politique de sécurité du contenu est absente, ce qui augmente le risque d\'attaques XSS.'
            })

        if not security_headers['X-XSS-Protection']:
            vulnerabilities.append({
                'id': 'XSS-01',
                'name': 'Protection XSS non activée',
                'severity': 'Modéré',
                'description': 'L\'en-tête X-XSS-Protection n\'est pas défini.'
            })

        # Add some random vulnerabilities for demo purposes
        if random.random() > 0.7:
            vulnerabilities.append({
                'id': 'INJ-01',
                'name': 'Vulnérabilité d\'injection potentielle',
                'severity': 'Critique',
                'description': 'Des points d\'injection potentiels ont été détectés.'
            })

        # Recommendations based on vulnerabilities
        recommendations = []
        for vuln in vulnerabilities:
            recommendations.append({
                'title': f'Corriger {vuln["name"]}',
                'description': f'Résoudre le problème: {vuln["description"]}',
                'priority': 'Haute' if vuln['severity'] == 'Critique' else 'Moyenne'
            })

        # Result object
        result = {
            'url': url,
            'scan_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'website_info': {
                'title': title_text,
                'name': title_text,
                'description': 'Site web analysé par WebSecure'
            },
            'risk_level_text': risk_level,
            'risk_score': risk_score,
            'vulnerability_count': len(vulnerabilities),
            'vulnerabilities': vulnerabilities,
            'security_headers': security_headers,
            'recommendations': recommendations
        }

        return result

    except Exception as e:
        print(f"Error analyzing website: {e}")
        return {
            'url': url,
            'scan_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'risk_level_text': 'Erreur',
            'risk_score': 0,
            'vulnerability_count': 1,
            'vulnerabilities': [{
                'id': 'ERROR-01',
                'name': 'Erreur d\'analyse',
                'severity': 'Critique',
                'description': f"Une erreur s'est produite lors de l'analyse: {str(e)}"
            }]
        }

# Special OPTIONS route for CORS preflight requests
@app.route('/api/health', methods=['OPTIONS'])
@app.route('/api/analyze', methods=['OPTIONS'])
@app.route('/api/reports', methods=['OPTIONS'])
@app.route('/api/reports/<path:result_id>', methods=['OPTIONS'])
def options_handler():
    response = make_response()
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
    response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE,OPTIONS')
    return response

# API Routes
@app.route('/api/health', methods=['GET'])
def health_check():
    return jsonify({
        'status': 'healthy',
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    })

@app.route('/api/analyze', methods=['POST'])
def analyze_website_route():
    data = request.get_json()

    if not data or not data.get('url'):
        return jsonify({'error': 'URL is required'}), 400

    try:
        # Analyze the website
        result = analyze_website(data['url'])

        # Generate a unique ID for the result
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        result_id = f"{data['url'].replace('://', '_').replace('/', '_').replace('.', '_')}_{timestamp}"

        # Save the result
        with open(f'reports/{result_id}.json', 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=4)

        # Return the response
        return jsonify({
            'status': 'success',
            'result_id': result_id,
            'summary': {
                'url': result['url'],
                'risk_level': result['risk_level_text'],
                'risk_score': result['risk_score'],
                'vulnerability_count': result['vulnerability_count']
            }
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/reports/<result_id>', methods=['GET'])
def get_report(result_id):
    try:
        with open(f'reports/{result_id}.json', 'r', encoding='utf-8') as f:
            report = json.load(f)
        return jsonify(report)
    except FileNotFoundError:
        return jsonify({'error': 'Report not found'}), 404

@app.route('/api/reports', methods=['GET'])
def list_reports():
    reports = []

    for filename in os.listdir('reports'):
        if filename.endswith('.json'):
            try:
                with open(f'reports/{filename}', 'r', encoding='utf-8') as f:
                    report = json.load(f)
                reports.append({
                    'id': filename.replace('.json', ''),
                    'url': report['url'],
                    'scan_date': report['scan_date'],
                    'risk_level': report['risk_level_text'],
                    'risk_score': report['risk_score']
                })
            except:
                continue

    # Sort by scan date (most recent first)
    reports.sort(key=lambda x: x['scan_date'], reverse=True)

    return jsonify(reports)

In [ ]:
# Run the Flask app using Google Colab's public URL feature
from google.colab import output
print("Starting server...")

# Start the Flask app
from threading import Thread
def run_flask():
    app.run(host='0.0.0.0', port=5000)

flask_thread = Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Get the public URL
print("Getting public URL...")
public_url = output.eval_js('google.colab.kernel.proxyPort(5000)')
print(f"Your public API URL is: {public_url}/api")
print(f"Update your api.js file with this URL!")
print("API_BASE_URL = '" + public_url + "/api'")

# Keep the notebook running
import time
print("Server is running. Keep this tab open. Press Ctrl+C to stop.")
while True:
    time.sleep(60)
    print(".", end="", flush=True)

Starting server...
Getting public URL...
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Your public API URL is: https://5000-m-s-203y6hpwtryt7-c.us-west1-0.prod.colab.dev/api
Update your api.js file with this URL!
API_BASE_URL = 'https://5000-m-s-203y6hpwtryt7-c.us-west1-0.prod.colab.dev/api'
Server is running. Keep this tab open. Press Ctrl+C to stop.
............................................................................................................................................................................................................................................................................................................